In [10]:
# filelist
import platform
import cv2
import numpy as np
import os
from pathlib import Path

root = os.getcwd() # Don't change this
data_dirname = '/data_test/plantvillage/' # Change as you like 
p = Path(root + data_dirname)
p.mkdir(exist_ok=True) 
if platform.system()=='Darwin':
  root = os.getcwd() # Don't change this
  data_dirname = '/data_test/plantvillage/' # Change as you like 
  p = Path(root + data_dirname)
  p.mkdir(exist_ok=True) 
#else:
#  p = Path("C:/Users/mwels/Documents/Uni/11. Semester/Deep learning in visual recognition/Plant_leave_diseases_dataset_without_augmentation")
#  p.mkdir(exist_ok=True)


classes = [
  'Apple___Apple_scab',
  'Apple___healthy',
  'Apple___Black_rot',
  'Apple___Cedar_apple_rust',
  "all"
  ]

if "all" in classes:
  classes = os.listdir(p)

for c in classes:
  print(c,end=" ")
  filelist = [x for x in (p/c).iterdir() if x.is_file()]
  for f in filelist:
    img = cv2.imread(str(f))
    if img is None:
      print(f'Failed to open {f}. Deleting file')
      os.remove(str(f))
# Pick first image of first class (i.e., cat)
# filelist = [x for x in (p/classes[0]).iterdir() if x.is_file()]
print(len(filelist))

Apple___Cedar_apple_rust Apple___Apple_scab Apple___Black_rot Apple___healthy 1645


In [11]:
from keras.layers import Dense,GlobalAveragePooling2D, Concatenate, Input, Flatten, Lambda, Layer
from keras.models import Model
from src import fenchel_young
from src import perturbations
import tensorflow as tf
#Custom ranking layer
class Ranking(Layer):
    def __init__(self, num_samples=1000000, sigma=0.1, noise='gumbel', batched=True):
        self.num_samples = num_samples
        self.sigma = sigma
        self.noise = noise
        self.batched=batched
        super(Ranking, self).__init__()

    def call(self, inputs):
        @perturbations.perturbed(num_samples=self.num_samples, sigma=self.sigma, noise=self.noise, batched=self.batched)
        def ranks_fn(x, axis=-1):
            return tf.cast(tf.argsort(tf.argsort(x, axis=axis), axis=axis), dtype=x.dtype)

        return ranks_fn(inputs)
#Split input layer
class TestL(Layer):
    def __init__(self, regularization_strength = 1):
        self.regularization_strength = regularization_strength
        super(TestL, self).__init__()

    def call(self, inputs):
        
        return tf.tensordot(inputs,tf.convert_to_tensor([1.2,2,3]),0)

ValueError: Existing Python class DType already has SerializedDType as its associated proto representation. Please ensure DType has a unique proto representation.

In [ ]:
Ranking()(Input(3))

<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'ranking_44')>

In [ ]:
from src import fenchel_young
from src import perturbations
#import gin
#@gin.configurable
#@perturbations.perturbed(num_samples=1000000, sigma=0.5, noise='gumbel', batched=True)
def ranks_fn(x, axis=-1):
  return tf.cast(tf.argsort(tf.argsort(x, axis=axis), axis=axis), dtype=x.dtype)

print(ranks_fn(tf.convert_to_tensor([[1.,0.2,3],[0.,1,2]])))

myloss = fenchel_young.FenchelYoungLoss(ranks_fn, num_samples=1000000, sigma=0.1, noise='gumbel', batched=True)
a = tf.convert_to_tensor([[1.,2,0],[0.,1,2]])
b = tf.convert_to_tensor([[0.5,1,0.2],[0.,1,1.2]])
print(myloss(a,b).numpy())
a = tf.convert_to_tensor([[1.,2,0],[0.,1,2]])
b = tf.convert_to_tensor([[1,2,0.2],[0.,1,1.2]])
print(myloss(a,b).numpy())

tf.Tensor(
[[1. 0. 2.]
 [0. 1. 2.]], shape=(2, 3), dtype=float32)
0.03224759
0.028181976


In [ ]:
from src.fast_soft_sort.tf_ops import soft_rank
class RankingLoss(tf.keras.losses.Loss):

    def call(self, y_true, y_pred):
        @tf.custom_gradient
        def loss(self,y_true, y_pred):
            diff = soft_rank(y_pred)-tf.cast(y_true,  dtype=y_pred.dtype)
    

            def grad(dy):
                dy = tf.reshape(dy, [tf.shape(dy)[0]] + (diff.shape.rank - 1) * [1])
                return dy*diff


            return 0.5*tf.square(diff)
        
        return self.loss(y_true, y_pred)

In [ ]:
from re import L
import tensorflow as tf

out = Input(3)
print(out)
Layer = TestL(0.1)(out)
print(Layer)
end = Dense(3)(Layer)
print(end)

KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name='input_50'), name='input_50', description="created by layer 'input_50'")
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 3), dtype=tf.float32, name=None), name='test_l_8/Tensordot:0', description="created by layer 'test_l_8'")
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 3), dtype=tf.float32, name=None), name='dense_35/BiasAdd:0', description="created by layer 'dense_35'")


In [ ]:
from keras.layers import Dense,GlobalAveragePooling2D, Concatenate, Input, Flatten, Lambda, Layer
from keras.models import Model

tilex = 2
number_of_tiles = tilex**2
tile_size = 255//tilex

from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input

conv_base = MobileNetV2(weights='imagenet',
                      include_top=False,
                      pooling='avg',
                      input_shape=(tile_size, tile_size, 3))


image_net = conv_base
tiles = Input((number_of_tiles,tile_size,tile_size,3))
layers = {}
embedds = {}
for i in range(number_of_tiles):
    layers[f'tile{i}'] = preprocess_input(Lambda(lambda x: x[:,i,:,:,:])(tiles))
    embedds[f'embedd{i}'] = image_net(layers[f'tile{i}'])

concatonation = Concatenate(axis=1)(list(embedds.values()))

out = Dense(500, activation="relu")(concatonation)
# out = Dense(500, activation="relu")(out)
out = Dense(number_of_tiles)(out)
out = Flatten()(out)
#out = Ranking()(out)

model = Model(inputs=tiles, outputs=out)

In [ ]:
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_76 (InputLayer)          [(None, 4, 127, 127  0           []                               
                                , 3)]                                                             
                                                                                                  
 lambda_146 (Lambda)            (None, 127, 127, 3)  0           ['input_76[0][0]']               
                                                                                                  
 lambda_147 (Lambda)            (None, 127, 127, 3)  0           ['input_76[0][0]']               
                                                                                                  
 lambda_148 (Lambda)            (None, 127, 127, 3)  0           ['input_76[0][0]']        

In [ ]:
total_num_layers = len(model.layers)
num_base_layers = len(conv_base.layers)
print(f"Total number of layers is {total_num_layers}")
print(f"Number of pretrained base layers is {num_base_layers}")

for layer in model.layers[:-4]:
    layer.trainable=False
for layer in model.layers[-4:]:
    layer.trainable=True

Total number of layers is 18
Number of pretrained base layers is 155


In [ ]:
class PartialRanksAccuracy(tf.keras.metrics.Mean):
  """This metric the proportion of matching ranks."""

  def __init__(self, name='partial_ranks_acc'):
    super().__init__(name=name)

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_pred = tf.cast(ranks_fn(y_pred, axis=-1), tf.int32)
    y_true = tf.cast(y_true, tf.int32)
    equals = tf.cast(y_true == y_pred, tf.float32)
    result = tf.math.reduce_mean(equals, axis=-1)
    super().update_state(
        tf.reduce_mean(result, axis=-1), sample_weight=sample_weight)

class ProjectedRanksAccuracy(tf.keras.metrics.Mean):
  """This metric is the normalized projection onto the permutahedron."""

  def __init__(self):
    super().__init__(name='projection_ranks_acc')

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_pred = tf.cast(ranks_fn(y_pred, axis=-1), tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    n = tf.cast(tf.shape(y_true)[-1], tf.float32)
    max_proj = (n - 1.0) * n * (2.0 * n - 1.0) / 6.0
    result = tf.math.reduce_sum(y_true * y_pred, axis=-1) / max_proj
    super().update_state(
        tf.reduce_mean(result, axis=-1), sample_weight=sample_weight)

In [ ]:
#NEW BETTER MODEL MSE
from keras import optimizers
from sklearn.model_selection import train_test_split
from src.permutation_tools import *
import tensorflow_ranking as tfr 

x_train, x_test = train_test_split(filelist)

train_generator = PermNetDataGenerator([x_train[0]]*256, 1,tilenumberx=tilex)
validation_generator = PermNetDataGenerator(x_test)


# Set up optimizer
sgd_optimizer = optimizers.SGD(learning_rate=0.001, decay=1e-6, momentum=0.9, nesterov=True)

# Compile model 
# softmax_loss = tfr.keras.losses.get(tfr.keras.losses.RankingLossKey.MEAN_SQUARED_LOSS)
model.compile(optimizer=sgd_optimizer,loss=tf.keras.losses.MeanSquaredError(), metrics=[ProjectedRanksAccuracy(), PartialRanksAccuracy()])

# Train model for 150 epochs
history = model.fit(train_generator,epochs=10)

Epoch 1/10
256/256 [==============================] - 79s 209ms/step - loss: 2.4893 - projection_ranks_acc: 0.6434 - partial_ranks_acc: 0.2373
Epoch 2/10
256/256 [==============================] - 50s 195ms/step - loss: 2.5417 - projection_ranks_acc: 0.6362 - partial_ranks_acc: 0.2402


KeyboardInterrupt: 

In [ ]:
#NEW BETTER MODEL
from keras import optimizers
from sklearn.model_selection import train_test_split
from src.permutation_tools import *
import tensorflow_ranking as tfr 

x_train, x_test = train_test_split(filelist)

train_generator = PermNetDataGenerator([x_train[0]]*512, 16, tilenumberx=tilex)
validation_generator = PermNetDataGenerator(x_test)


# Set up optimizer
sgd_optimizer = optimizers.SGD(learning_rate=0.001, decay=1e-6, momentum=0.9, nesterov=True)

# Compile model 
# softmax_loss = tfr.keras.losses.get(tfr.keras.losses.RankingLossKey.MEAN_SQUARED_LOSS)
model.compile(optimizer=sgd_optimizer,loss=myloss, metrics=[ProjectedRanksAccuracy(), PartialRanksAccuracy()])

# Train model for 150 epochs
history = model.fit(train_generator,epochs=150)

Epoch 1/150
32/32 [==============================] - 118s 3s/step - loss: 97.2108 - projection_ranks_acc: 0.6239 - partial_ranks_acc: 0.2251
Epoch 2/150
32/32 [==============================] - 99s 3s/step - loss: 81.5335 - projection_ranks_acc: 0.6508 - partial_ranks_acc: 0.2593
Epoch 3/150
32/32 [==============================] - 83s 3s/step - loss: 82.6012 - projection_ranks_acc: 0.6609 - partial_ranks_acc: 0.2510
Epoch 4/150
32/32 [==============================] - 107s 3s/step - loss: 82.6795 - projection_ranks_acc: 0.6295 - partial_ranks_acc: 0.2441
Epoch 5/150
32/32 [==============================] - 105s 3s/step - loss: 81.1521 - projection_ranks_acc: 0.6582 - partial_ranks_acc: 0.2690
Epoch 6/150
12/32 [==========>...................] - ETA: 1:00 - loss: 82.0368 - projection_ranks_acc: 0.6209 - partial_ranks_acc: 0.2591


KeyboardInterrupt



In [ ]:
tesgen = PermNetDataGenerator([x_train[0]]*512, 16, tilenumberx=tilex)
X, y = tesgen.next()
print(tesgen.input_shape)
print(X.shape)
showPermImg(X[0], y[0], 2)
ranks_fn(model.predict(np.expand_dims(X[0], 0)))

NameError: name 'PermNetDataGenerator' is not defined

In [ ]:
from keras import optimizers
from sklearn.model_selection import train_test_split
from src.permutation_tools import *
import tensorflow_ranking as tfr 

x_train, x_test = train_test_split(filelist)

train_generator = PermNetDataGenerator(x_train)
validation_generator = PermNetDataGenerator(x_test)


# Set up optimizer
sgd_optimizer = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)

# Compile model 
# softmax_loss = tfr.keras.losses.get(tfr.keras.losses.RankingLossKey.MEAN_SQUARED_LOSS)
model.compile(optimizer=sgd_optimizer,loss=myloss, metrics=['binary_accuracy'])

step_size_train = train_generator.n//train_generator.batch_size # Number of mini-batches per epoch (training)
step_size_val = validation_generator.n//validation_generator.batch_size # Number of mini-batches per epoch (validation)

# Train model for 15 epochs
history = model.fit_generator(generator=train_generator,
                   validation_data=validation_generator,
                   validation_steps=step_size_val,
                   steps_per_epoch=step_size_train,
                   epochs=15)

/var/folders/l9/9d49h4t14s52dq6c276zwhgc0000gn/T/ipykernel_25950/2725868492.py:23: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_generator,


Epoch 1/15


KeyboardInterrupt: 

In [ ]:
from keras import optimizers
from sklearn.model_selection import train_test_split
from src.permutation_tools import *
import tensorflow_ranking as tfr 

x_train, x_test = train_test_split(filelist)

train_generator = PermNetDataGenerator(x_train)
validation_generator = PermNetDataGenerator(x_test)


# Set up optimizer
sgd_optimizer = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)

# Compile model 
# softmax_loss = tfr.keras.losses.get(tfr.keras.losses.RankingLossKey.MEAN_SQUARED_LOSS)
model.compile(optimizer=sgd_optimizer,loss=tfr.keras.losses.MeanSquaredLoss(), metrics=['accuracy'])

step_size_train = train_generator.n//train_generator.batch_size # Number of mini-batches per epoch (training)
step_size_val = validation_generator.n//validation_generator.batch_size # Number of mini-batches per epoch (validation)

# Train model for 15 epochs
history = model.fit_generator(generator=train_generator,
                   validation_data=validation_generator,
                   validation_steps=step_size_val,
                   steps_per_epoch=step_size_train,
                   epochs=15)


In [ ]:
newDataGen = PermNetDataGenerator(input=x_train, batch_size=4, preprocess_func=preprocess_input)

In [ ]:
X, y = next(newDataGen)

In [ ]:
showPermImg(X[0],y[0])

In [ ]:
test_gen = PermNetDataGenerator(x_test) # never do this
X, y = next(test_gen)
model.predict(np.expand_dims(X[0], axis=0))

tensorflow_ranking.python.keras.losses.ListMLELoss